In [ ]:
import qiskit
from qiskit import Aer
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.aqua.operators import Z2Symmetries, WeightedPauliOperator
from qiskit.aqua.algorithms import VQE
from qiskit.aqua import QuantumInstance

# Define the molecular information for BeH2
molecule = 'Be . H . H\n' + 'H 1 {r}\n' + 'H 1 {r} 2 {theta}\n'
distance = 1.4  # in Angstroms
angle = 104.5  # in degrees
molecule = molecule.format(r=distance, theta=angle)
charge = 0
spin = 0

# Compute the molecular integrals and construct the WeightedPauliOperator
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry.core import Hamiltonian, QubitMappingType

driver = PySCFDriver(atom=molecule, charge=charge, spin=spin, basis='sto3g')
qmolecule = driver.run()
operator = Hamiltonian(qubit_mapping=QubitMappingType.PARITY, two_qubit_reduction=True, freeze_core=True,
                       orbital_reduction=[-3, -2])
operator.from_input_string(qmolecule.one_body_integrals, qmolecule.two_body_integrals)

# Exploit the symmetries of the problem
symmetries = Z2Symmetries.find_Z2_symmetries(operator)
num_qubits = operator.num_qubits

# Define the optimizer and variational form for the VQE
optimizer = SPSA(maxiter=1000)
var_form = RYRZ(num_qubits, depth=3, entanglement='linear')

# Construct the VQE algorithm and run it
vqe = VQE(operator, var_form, optimizer, quantum_instance=QuantumInstance(Aer.get_backend('statevector_simulator')))
result = vqe.run()

# Print the ground state energy and the optimized parameters
print(f"Ground state energy: {result.eigenvalue.real:.4f}")
print(f"Optimized parameters: {result.optimal_point}")
